# Process the original file to match function

In [1]:
# to auto-reload the imports
# if we change something in our functions
%load_ext autoreload
%autoreload 2

%load_ext nb_black

# import the libraries required to do the work
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
import pandas as pd
import seaborn as sns
import datetime
from scipy.stats import norm
from scipy.interpolate import interp1d
import os
from tqdm.notebook import tqdm
import re

from src.utils.sharepoint import get_T1_ren_6kPax_schedule
from decouple import AutoConfig
from pathlib import Path

<IPython.core.display.Javascript object>

In [2]:
# get the schedule from Sharepoint
# app key expired for SharePoint
# get_T1_ren_6kPax_schedule()

# get the paths to config (could be made as a function for notebooks)

DOTENV_FILE_PATH = Path(os.getcwd()) / "../../../data/secret/.env"
config = AutoConfig(search_path=DOTENV_FILE_PATH)

path_relative = config("T1_ren_6kPax_schedule_path")

path_data = Path(os.getcwd()) / ".." / ".." / ".." / path_relative

<IPython.core.display.Javascript object>

In [3]:
# define a function to insert space in flight number
def insert_space_after_letters(test_str: str):
    res = re.sub("[A-Za-z]+", lambda ele: ele[0] + " ", test_str)
    return res

<IPython.core.display.Javascript object>

In [4]:
# import the schedule from the excel file produced by Aero department
# data_arr = pd.read_excel(
#     path_data,
#     header=0,
#     sheet_name="ARR",
# )

data_dep = pd.read_excel(
    path_data,
    header=0,
    sheet_name="DEP",
)



<IPython.core.display.Javascript object>

In [5]:
data_dep

,運航者,航空機識別,型式,登録記号,目的地,ｽﾎﾟｯﾄ番号,DI区分,貨客区分,目的,定期区分,...,ICAO Code,座席数,重量,搭乗者数,L/F,ピーク時整理用,大型化,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,NCA,NCA283,B744,JA05KZ,WSSS,214,I,C,SI,S,...,E,-,-,-,-,2017-03-18 00:00:00,NaN,NaN,NaN,NaN
1,TAX,TAX611,A333,HSXTF,VTBD,39,I,P,SI,S,...,E,377,235,348.65,0.924801,2017-03-18 00:00:00,NaN,NaN,NaN,NaN
2,ANA,ANA8559,B763,JA8323,ROAH,212,D,C,SD,S,...,D,-,-,-,-,2017-03-18 00:00:00,NaN,NaN,NaN,NaN
3,JAL,JAL727,B788,JA830J,VTBS,31,I,P,SI,S,...,E,186,200,173.85,0.934677,2017-03-18 00:00:00,NaN,NaN,NaN,NaN
4,FDX,FDX96,B77L,N869FD,ZSPD,252,I,C,SI,S,...,E,-,-,-,-,2017-03-18 01:00:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,NEW,NEW128,A333,NEW128,xxxx,CG9,I,P,SI,S,...,E,NaN,NaN,266,-,2017-03-19 14:00:00,NaN,NaN,NaN,NaN
649,NEW,NEW129,A320,NEW129,xxxx,CG9,I,P,SI,S,...,C,NaN,NaN,147.25,-,2017-03-19 16:00:00,NaN,NaN,NaN,NaN
650,NEW,NEW130,A333,NEW130,xxxx,CG9,I,P,SI,S,...,E,NaN,NaN,266,-,2017-03-19 19:00:00,NaN,NaN,NaN,NaN
651,NEW,NEW131,A320,NEW131,xxxx,CG9,I,P,SI,S,...,C,NaN,NaN,147.25,-,2017-03-19 21:00:00,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [6]:
# edit data to match application format
data_arr_processed = data_arr.copy()
data_arr_processed["A/D"] = "A"
data_arr_processed["T1/T2(MM/9C/7C/TW)"] = "T1"
data_arr_processed["Int'l Regions"] = "unknown"

dct_name_change = {
    "貨客区分": "Category(P/C/O)",
    "DI区分": "Sector",
    "航空機識別": "Flight Number",  # <- to be split with space between letters and digits
    "座席数": "SEATS FC",
    "搭乗者数": "Pax_SUM FC",
    "到着予定日": "Flight Date",
    "STA": "Scheduled Time",
}

data_arr_processed.rename(columns=dct_name_change, inplace=True)

data_arr_processed["Flight Number"] = data_arr_processed["Flight Number"].apply(
    insert_space_after_letters
)

NameError: name 'data_arr' is not defined

<IPython.core.display.Javascript object>

In [7]:
# problem : no Seats and Pax number for arrival flights...
# keep only relevant columns
# data_arr_processed = data_arr_processed[
#    [
#        "A/D",
#        "T1/T2(MM/9C/7C/TW)",
#        "Int'l Regions",
#        "Category(P/C/O)",
#        "Sector",
#        "Flight Number",
#        "SEATS FC",
#        "Pax_SUM FC",
#        "Flight Date",
#        "Scheduled Time",
#    ]
# ]

<IPython.core.display.Javascript object>

In [8]:
# edit data to match application format
data_dep_processed = data_dep.copy()

data_dep_processed["A/D"] = "D"
data_dep_processed["T1/T2(MM/9C/7C/TW)"] = "T1"
data_dep_processed["Intl Regions"] = "unknown"

dct_name_change = {
    "貨客区分": "Category(P/C/O)",
    "DI区分": "Sector",
    "航空機識別": "Flight Number",  # <- to be split with space between letters and digits
    "座席数": "SEATS FC",
    "搭乗者数": "PAX_SUM FC",
    "出発予定日": "Flight Date",
    "STD": "Scheduled Time",
}

data_dep_processed.rename(columns=dct_name_change, inplace=True)


<IPython.core.display.Javascript object>

In [10]:

data_dep_processed["Flight Number"] = data_dep_processed["Flight Number"].apply(
    insert_space_after_letters
)

<IPython.core.display.Javascript object>

In [11]:
# keep only relevant columns
data_dep_processed = data_dep_processed[
    [
        "A/D",
        "T1/T2(MM/9C/7C/TW)",
        "Intl Regions",
        "Category(P/C/O)",
        "Sector",
        "Flight Number",
        "SEATS FC",
        "PAX_SUM FC",
        "Flight Date",
        "Scheduled Time",
    ]
]


<IPython.core.display.Javascript object>

In [12]:
data_dep_processed

,A/D,T1/T2(MM/9C/7C/TW),Intl Regions,Category(P/C/O),Sector,Flight Number,SEATS FC,PAX_SUM FC,Flight Date,Scheduled Time
0,D,T1,unknown,C,I,NCA 283,-,-,2017-03-18,00:05:00
1,D,T1,unknown,P,I,TAX 611,377,348.65,2017-03-18,00:10:00
2,D,T1,unknown,C,D,ANA 8559,-,-,2017-03-18,00:05:00
3,D,T1,unknown,P,I,JAL 727,186,173.85,2017-03-18,00:40:00
4,D,T1,unknown,C,I,FDX 96,-,-,2017-03-18,01:10:00
...,...,...,...,...,...,...,...,...,...,...
648,D,T1,unknown,P,I,NEW 128,NaN,266,2017-03-19,14:30:00
649,D,T1,unknown,P,I,NEW 129,NaN,147.25,2017-03-19,16:40:00
650,D,T1,unknown,P,I,NEW 130,NaN,266,2017-03-19,19:30:00
651,D,T1,unknown,P,I,NEW 131,NaN,147.25,2017-03-19,21:20:00


<IPython.core.display.Javascript object>

In [13]:
type(data_dep_processed["Scheduled Time"][0])

datetime.time

<IPython.core.display.Javascript object>

In [15]:
data_dep_processed['datetime'] = pd.to_datetime(data_dep_processed['Flight Date'].astype('str') + ' '  +data_dep_processed['Scheduled Time'].astype('str'))

<IPython.core.display.Javascript object>

In [16]:
type(data_dep_processed['datetime'][0])

pandas._libs.tslibs.timestamps.Timestamp

<IPython.core.display.Javascript object>

In [17]:
data_dep_processed[mask_date].set_index('datetime')['PAX_SUM FC'].replace('-',value=0).resample('60min').agg('sum').plot()

NameError: name 'mask_date' is not defined

<IPython.core.display.Javascript object>

In [20]:

# correct input mistake (?)
mask_replace = data_dep_processed["PAX_SUM FC"] == "-"
data_dep_processed.loc[mask_replace, "PAX_SUM FC"] = 0

mask_replace = data_dep_processed["SEATS FC"] == "-"
data_dep_processed.loc[mask_replace, "SEATS FC"] = 0


<IPython.core.display.Javascript object>

In [22]:
# fill in the seats number where missing
def custom_func_seats(pax):
    if pax == 147.25:
        seats = 180
    if pax == 266:
        seats = 285
    return seats


mask_NEW_no_seats = (
    data_dep_processed["Flight Number"].apply(lambda x: x.split(" ")[0]) == "NEW"
) & (data_dep_processed["SEATS FC"].apply(lambda x: np.isnan(x)))

data_dep_processed.loc[mask_NEW_no_seats, "SEATS FC"] = data_dep_processed.loc[
    mask_NEW_no_seats, "PAX_SUM FC"
].apply(lambda x: custom_func_seats(x))

<IPython.core.display.Javascript object>

In [ ]:
# we should add a step to merge the two in one
# as we do not have seats and Pax for T1 schedule, let's forget about it for now

In [23]:
output_path = (
    Path(os.getcwd())
    / "../../../data/processed/Schedule (30th terminal peak, 6000 pax)_PROCESSED.xlsx"
)

writer = pd.ExcelWriter(
    output_path,
)

with writer as writer:
    data_dep_processed.to_excel(writer, sheet_name="schedule")

<IPython.core.display.Javascript object>